In [2]:
import zipfile
import os

# Name of zip file that will be downloaded.
#  The Kaggle API command will download specific zip name.
#  Usually matching the last part of the API command.
zip_name = "fake-news-detection.zip"
# Check if local machine has this zip file already in location of notebook.
# If it does not, it downloads it.
if not os.path.exists(zip_name):
    # Kaggle API command.
    !kaggle datasets download -d jruvika/fake-news-detection

# Name of folder that csv files will export to.
folder_name = "Resources"
# Check if local machine has this folder created already in location of notebook.
# If it does not, it creates it.
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
directory_to_extract_to = folder_name

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)



  0%|          | 0.00/4.89M [00:00<?, ?B/s]
 20%|##        | 1.00M/4.89M [00:00<00:00, 7.24MB/s]
 61%|######1   | 3.00M/4.89M [00:00<00:00, 10.6MB/s]
100%|##########| 4.89M/4.89M [00:00<00:00, 13.8MB/s]
100%|##########| 4.89M/4.89M [00:00<00:00, 12.5MB/s]


In [7]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

csv_path = "Resources/data.csv"
data_df = pd.read_csv(csv_path)

data_df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [8]:
print(data_df.columns)
print(data_df.shape)
print(data_df.isnull().sum())

data_df = data_df.fillna('')

Index(['URLs', 'Headline', 'Body', 'Label'], dtype='object')
(4009, 4)
URLs         0
Headline     0
Body        21
Label        0
dtype: int64


In [9]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [52]:
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
# word_tokenize accepts
# a string as an input, not a file.
stop_words = set(stopwords.words('english'))
file1 = open("data.csv", errors="ignore")
 
# Use this to read file content as a stream:
line = file1.read()
words = line.split()
for r in words:
    if not r in stop_words:
        appendFile = open('filteredtext.txt','a')
        appendFile.write(" "+r)
        appendFile.close()

In [53]:
data_df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [54]:
dict = { 1 : 'REAL' , 0 : 'FAKE'}
label = data_df['Label'].map(dict)

In [55]:
label

0       REAL
1       REAL
2       REAL
3       REAL
4       REAL
        ... 
4004    FAKE
4005    FAKE
4006    FAKE
4007    REAL
4008    FAKE
Name: Label, Length: 4009, dtype: object

In [56]:
x_df = data_df['Body']
y_df = data_df['Label']

In [57]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
count_vectorizer = CountVectorizer()

In [59]:
count_vectorizer.fit_transform(x_df)

<4009x46069 sparse matrix of type '<class 'numpy.int64'>'
	with 926875 stored elements in Compressed Sparse Row format>

In [60]:
freq_term_matrix = count_vectorizer.transform(x_df)

In [61]:
tfidf = TfidfTransformer(norm = "l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [62]:
print(tf_idf_)matrix

  (0, 45780)	0.0177392673856147
  (0, 45772)	0.013186156318419455
  (0, 45686)	0.009956626261304169
  (0, 45501)	0.034427846548055425
  (0, 45408)	0.024508260437741354
  (0, 45388)	0.019764621024298108
  (0, 45372)	0.01814381264583244
  (0, 45256)	0.04436397068404117
  (0, 45182)	0.023751664869733624
  (0, 45117)	0.03226933565557967
  (0, 45036)	0.044394548402472846
  (0, 45010)	0.008086367605233595
  (0, 44984)	0.07744156577016219
  (0, 44943)	0.025631665705911605
  (0, 44933)	0.009189128835497664
  (0, 44910)	0.036998359520545664
  (0, 44858)	0.018263076671834477
  (0, 44844)	0.009201159894997297
  (0, 44840)	0.01502752098413376
  (0, 44687)	0.008417017961555969
  (0, 44663)	0.011967290463513741
  (0, 44598)	0.01738329250947253
  (0, 44585)	0.02170484686559111
  (0, 44521)	0.016308795816510017
  (0, 44325)	0.025060814402536758
  :	:
  (4008, 3915)	0.034020193599148846
  (4008, 3811)	0.015983410501801198
  (4008, 3711)	0.009080531283338885
  (4008, 3464)	0.04266961959233722
  (4008, 3

# Modeling

In [63]:
# Splitting the data into test data and train data
x_train, x_test, y_train, y_test = train_test_split(tf_idf_matrix,y_df, random_state=0)

In [64]:
y_df.value_counts()

0    2137
1    1872
Name: Label, dtype: int64

# Logistic Regression

In [128]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Accuracy = logreg.score(x_test, y_test)

print(Accuracy*100)

97.30807577268196


# Naive-Bayes

In [129]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(x_train, y_train)
Accuracy = NB.score(x_test, y_test)

print(Accuracy*100)

93.91824526420737


# Decision Tree

In [130]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
Accuracy = clf.score(x_test, y_test)

print(Accuracy*100)

95.81256231306082


# Passive-Aggressive Classifier

In [131]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(x_train,y_train)

#Predict on the test set and calculate accuracy
y_pred_pa=pac.predict(x_test)
score=accuracy_score(y_test,y_pred_pa)

print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 98.31%


In [132]:
# make predictions--> Logistic Regression
predictions = logreg.predict(tf_idf_matrix)
# Submissions
test_text = data_df.Headline
log_predict = pd.DataFrame({'headline' : test_text, 'logistic_regression' : predictions})

# make predictions--> Naive Bayes
predictions2 = NB.predict(tf_idf_matrix)
# Submissions
NB_predict = pd.DataFrame({'naive_bayes' : predictions2})
log_predict = pd.concat([log_predict, NB_predict], axis = 1)

# make predictions--> Decision Tree
predictions3 = clf.predict(tf_idf_matrix)
# Submissions
clf_predict = pd.DataFrame({'decision_tree' : predictions3})
log_predict = pd.concat([log_predict, clf_predict], axis = 1)


# make predictions--> Passive Aggressive Classifier
predictions4 = pac.predict(tf_idf_matrix)
# Submissions
pac_predict = pd.DataFrame({'passive_aggressive_classifier' : predictions4})
log_predict = pd.concat([log_predict, pac_predict], axis = 1)
log_predict.head(50)

,headline,logistic_regression,naive_bayes,decision_tree,passive_aggressive_classifier
0,Four ways Bob Corker skewered Donald Trump,1,1,1,1
1,Linklater's war veteran comedy speaks to moder...,1,1,1,1
2,Trump’s Fight With Corker Jeopardizes His Legi...,1,1,1,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,1,1,1,1
4,Jason Aldean opens 'SNL' with Vegas tribute,1,1,1,1
5,JetNation FanDuel League; Week 4,0,0,0,0
6,Kansas Tried a Tax Plan Similar to Trump’s. It...,1,1,1,1
7,"India RBI chief: growth important, but not at ...",1,1,1,1
8,EPA chief to sign rule on Clean Power Plan exi...,1,1,1,1
9,Talks on sale of Air Berlin planes to easyJet ...,1,1,1,1


In [136]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import pandas as pd


username = "postgres"
password = "group4winners"
server = "finalprojgroup4.c26jlhodxytp.us-east-2.rds.amazonaws.com"
database = "FinalProjGroup4"
# create engine to police-violence-racial-equity postgres
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{server}/{database}')
# Create our session (link) from Python to the DB
session = Session(engine)
table_name = "prediction_data"
log_predict.to_sql(table_name, con=engine, if_exists='append')
# Close Session
session.close()